# Master's thesis - Lukas Meuris - graphCast training - loss functions

This notebook contains the code to train the graphCast model. 
We use this notebook to traint he graphcast model with different loss-functions to see the influence of it on the model performance.

 

In [ ]:
# @title Pip install graphcast and dependencies

#!pip install --upgrade https://github.com/deepmind/graphcast/archive/master.zip


# Installation and initialisation.


In [23]:
# @title Imports
import sys
sys.path.append("../")

import dataclasses
import functools

from google.cloud import storage
from graphcast import autoregressive
from graphcast import casting
from graphcast import data_utils
from graphcast import graphcast
from graphcast import rollout
from graphcast import normalization
from graphcast import xarray_jax
from graphcast import xarray_tree
import haiku as hk
import jax
import numpy as np
import xarray

import os
import time
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'

def parse_file_parts(file_name):
  return dict(part.split("-", 1) for part in file_name.split("_"))


# Access data from GCS

In [24]:
# @title Authenticate with Google Cloud Storage

gcs_client = storage.Client.create_anonymous_client()
gcs_bucket = gcs_client.get_bucket("dm_graphcast")

# Load the Data and initialize the model

## Load the model parameters

We use random parameters for the model initialization. 
We'll get random predictions, but we can change the model architecture.


*Checkpoints vary across a few axes:*
- *The mesh size specifies the internal graph representation of the earth. Smaller meshes will run faster but will have worse outputs. The mesh size does not affect the number of parameters of the model.*
- *The resolution and number of pressure levels must match the data. Lower resolution and fewer levels will run a bit faster. Data resolution only affects the encoder/decoder.*
- *All our models predict precipitation. However, ERA5 includes precipitation, while HRES does not. Our models marked as "ERA5" take precipitation as input and expect ERA5 data as input, while model marked "ERA5-HRES" do not take precipitation as input and are specifically trained to take HRES-fc0 as input (see the data section below).*


In [25]:
# @title choose model parameters
random_mesh_size = 5 # mesh size: 4 - 6
random_gnn_msg_steps = 16 # message passing steps: 1 - 32
random_latent_size = 512 # latent size: 16,32,64,128,256,512
random_levels = 13 # levels: 13 or 37


In [26]:
# @title load the model parameters
params = None  # Filled in below
state = {}
model_config = graphcast.ModelConfig(
    resolution=0,
    mesh_size=random_mesh_size,
    latent_size=random_latent_size,
    gnn_msg_steps=random_gnn_msg_steps,
    hidden_layers=1,
    radius_query_fraction_edge_length=0.6)
task_config = graphcast.TaskConfig(
    input_variables=graphcast.TASK.input_variables,
    target_variables=graphcast.TASK.target_variables,
    forcing_variables=graphcast.TASK.forcing_variables,
    pressure_levels=graphcast.PRESSURE_LEVELS[random_levels],
    input_duration=graphcast.TASK.input_duration,
)
model_config

ModelConfig(resolution=0, mesh_size=5, latent_size=512, gnn_msg_steps=16, hidden_layers=1, radius_query_fraction_edge_length=0.6, mesh2grid_edge_normalization_factor=None)

# Load the ERA5 data


In [49]:
# @title Get and filter the list of available example datasets, then select one.

# Define the relative path to the file
relative_path = "ERA5_data/obs_data.zarr"

# Get the absolute path by joining the current directory with the relative path
absolute_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
print(absolute_path)

# Open the Zarr file using xarray
obs_data = xarray.open_zarr(absolute_path)

/home/jupyter-lukas/Masters-Thesis/ERA5_data/obs_data.zarr


In [50]:
# view obs_data:
obs_data

<xarray.Dataset> Size: 63GB
Dimensions:                       (batch: 1, time: 92044, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
    datetime                      (batch, time) datetime64[ns] 736kB dask.array<chunksize=(1, 23011), meta=np.ndarray>
  * lat                           (lat) float64 256B -87.19 -81.56 ... 87.19
  * level                         (level) int64 104B 50 100 150 ... 850 925 1000
  * lon                           (lon) float64 512B 0.0 5.625 ... 348.8 354.4
  * time                          (time) datetime64[ns] 736kB 1959-01-01 ... ...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 754MB dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 10GB dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>

## extract the training data

In [30]:

#time slice: 1980-01-01T00:00:00.000000000 to 2019-12-31T00:00:00.000000000 - TRAINING
train_data = obs_data.sel(time=slice('1980-01-01T00:00:00.000000000','2019-12-31T00:00:00.000000000'))
train_data

<xarray.Dataset> Size: 40GB
Dimensions:                       (batch: 1, time: 58437, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
    datetime                      (batch, time) datetime64[ns] 467kB dask.array<chunksize=(1, 15342), meta=np.ndarray>
  * lat                           (lat) float64 256B -87.19 -81.56 ... 87.19
  * level                         (level) int64 104B 50 100 150 ... 850 925 1000
  * lon                           (lon) float64 512B 0.0 5.625 ... 348.8 354.4
  * time                          (time) datetime64[ns] 467kB 1980-01-01 ... ...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 479MB dask.array<chunksize=(1, 20, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 479MB dask.array<chunksize=(1, 20, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 479MB dask.array<chunksize=(1, 20, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 6GB dask.array<chunksize=(1, 20, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 6GB dask.array<chunksize=(1, 20, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 479MB dask.array<chunksize=(1, 20, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 479MB dask.array<chunksize=(1, 20, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 6GB dask.array<chunksize=(1, 20, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 6GB dask.array<chunksize=(1, 20, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 6GB dask.array<chunksize=(1, 20, 13, 64, 32), meta=np.ndarray>

## extract the evaluation data

In [32]:

#time slice: 2020-01-01T00:00:00.000000000 to 2022-01-01T00:00:00.000000000 - EVALUATION
eval_data = obs_data.sel(time=slice('2020-01-01T00:00:00.000000000','2021-01-01T00:00:00.000000000'))
eval_data

<xarray.Dataset> Size: 1GB
Dimensions:                       (batch: 1, time: 1465, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
    datetime                      (batch, time) datetime64[ns] 12kB dask.array<chunksize=(1, 1465), meta=np.ndarray>
  * lat                           (lat) float64 256B -87.19 -81.56 ... 87.19
  * level                         (level) int64 104B 50 100 150 ... 850 925 1000
  * lon                           (lon) float64 512B 0.0 5.625 ... 348.8 354.4
  * time                          (time) datetime64[ns] 12kB 2020-01-01 ... 2...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 12MB dask.array<chunksize=(1, 80, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 12MB dask.array<chunksize=(1, 80, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 12MB dask.array<chunksize=(1, 80, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 156MB dask.array<chunksize=(1, 80, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 8kB dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 156MB dask.array<chunksize=(1, 80, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 12MB dask.array<chunksize=(1, 80, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 12MB dask.array<chunksize=(1, 80, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 156MB dask.array<chunksize=(1, 80, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 156MB dask.array<chunksize=(1, 80, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 156MB dask.array<chunksize=(1, 80, 13, 64, 32), meta=np.ndarray>

## choose number of training and evaluation steps.

In [33]:
# @title Choose training and eval data to extract
train_steps = 12  # {1 - obs_data.sizes["time"]-2} | 12 = 3days
eval_steps = 40 # {1 - obs_data.sizes["time"]-2} | 40 = 10days

# !test: making data similar to graphCast example_batch

In [39]:
example_batch = eval_data.isel(time=slice(0,42))
example_batch = example_batch.compute()

## extract training and eval inputs, targets and forcings.

In [40]:

train_inputs, train_targets, train_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{train_steps*6}h"),
    **dataclasses.asdict(task_config))

eval_inputs, eval_targets, eval_forcings = data_utils.extract_inputs_targets_forcings(
    example_batch, target_lead_times=slice("6h", f"{eval_steps*6}h"),
    **dataclasses.asdict(task_config))

print("All data:  ", example_batch.dims.mapping)
print("Train Inputs:  ", train_inputs.dims.mapping)
print("Train Targets: ", train_targets.dims.mapping)
print("Train Forcings:", train_forcings.dims.mapping)
print("Eval Inputs:   ", eval_inputs.dims.mapping)
print("Eval Targets:  ", eval_targets.dims.mapping)
print("Eval Forcings: ", eval_forcings.dims.mapping)

All data:   {'batch': 1, 'time': 42, 'lon': 64, 'lat': 32, 'level': 13}
Train Inputs:   {'batch': 1, 'time': 2, 'lon': 64, 'lat': 32, 'level': 13}
Train Targets:  {'batch': 1, 'time': 12, 'lon': 64, 'lat': 32, 'level': 13}
Train Forcings: {'batch': 1, 'time': 12, 'lon': 64, 'lat': 32}
Eval Inputs:    {'batch': 1, 'time': 2, 'lon': 64, 'lat': 32, 'level': 13}
Eval Targets:   {'batch': 1, 'time': 40, 'lon': 64, 'lat': 32, 'level': 13}
Eval Forcings:  {'batch': 1, 'time': 40, 'lon': 64, 'lat': 32}


## Load normalization data

In [41]:

with gcs_bucket.blob("stats/diffs_stddev_by_level.nc").open("rb") as f:
    diffs_stddev_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob("stats/mean_by_level.nc").open("rb") as f:
    mean_by_level = xarray.load_dataset(f).compute()
with gcs_bucket.blob("stats/stddev_by_level.nc").open("rb") as f:
    stddev_by_level = xarray.load_dataset(f).compute()


## Build jitted functions, and possibly initialize random weights

In [43]:

def construct_wrapped_graphcast(
    model_config: graphcast.ModelConfig,
    task_config: graphcast.TaskConfig):
  """Constructs and wraps the GraphCast Predictor."""
  # Deeper one-step predictor.
  predictor = graphcast.GraphCast(model_config, task_config)

  # Modify inputs/outputs to `graphcast.GraphCast` to handle conversion to
  # from/to float32 to/from BFloat16.
  predictor = casting.Bfloat16Cast(predictor)

  # Modify inputs/outputs to `casting.Bfloat16Cast` so the casting to/from
  # BFloat16 happens after applying normalization to the inputs/targets.
  predictor = normalization.InputsAndResiduals(
      predictor,
      diffs_stddev_by_level=diffs_stddev_by_level,
      mean_by_level=mean_by_level,
      stddev_by_level=stddev_by_level)

  # Wraps everything so the one-step model can produce trajectories.
  predictor = autoregressive.Predictor(predictor, gradient_checkpointing=True)
  return predictor


@hk.transform_with_state
def run_forward(model_config, task_config, inputs, targets_template, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  return predictor(inputs, targets_template=targets_template, forcings=forcings)


@hk.transform_with_state
def loss_fn(model_config, task_config, inputs, targets, forcings):
  predictor = construct_wrapped_graphcast(model_config, task_config)
  loss, diagnostics = predictor.loss(inputs, targets, forcings)
  return xarray_tree.map_structure(
      lambda x: xarray_jax.unwrap_data(x.mean(), require_jax=True),
      (loss, diagnostics))

def grads_fn(params, state, model_config, task_config, inputs, targets, forcings):
  def _aux(params, state, i, t, f):
    (loss, diagnostics), next_state = loss_fn.apply(
        params, state, jax.random.PRNGKey(0), model_config, task_config,
        i, t, f)
    return loss, (diagnostics, next_state)
  (loss, (diagnostics, next_state)), grads = jax.value_and_grad(
      _aux, has_aux=True)(params, state, inputs, targets, forcings)
  return loss, diagnostics, next_state, grads

# Jax doesn't seem to like passing configs as args through the jit. Passing it
# in via partial (instead of capture by closure) forces jax to invalidate the
# jit cache if you change configs.
def with_configs(fn):
  return functools.partial(
      fn, model_config=model_config, task_config=task_config)

# Always pass params and state, so the usage below are simpler
def with_params(fn):
  return functools.partial(fn, params=params, state=state)

# Our models aren't stateful, so the state is always empty, so just return the
# predictions. This is required by our rollout code, and generally simpler.
def drop_state(fn):
  return lambda **kw: fn(**kw)[0]

init_jitted = jax.jit(with_configs(run_forward.init))

if params is None:
  params, state = init_jitted(
      rng=jax.random.PRNGKey(0),
      inputs=train_inputs,
      targets_template=train_targets,
      forcings=train_forcings)

loss_fn_jitted = drop_state(with_params(jax.jit(with_configs(loss_fn.apply))))
grads_fn_jitted = with_params(jax.jit(with_configs(grads_fn)))
run_forward_jitted = drop_state(with_params(jax.jit(with_configs(
    run_forward.apply))))

In [44]:
# @title config check

assert model_config.resolution in (0, 360. / eval_inputs.sizes["lon"]), (
  "Model resolution doesn't match the data resolution. You likely want to "
  "re-filter the dataset list, and download the correct data.")


# Train the model

The following operations require a large amount of memory and, depending on the accelerator being used, will only fit the very small "random" model on low resolution data. It uses the number of training steps selected above.

The first time executing the cell takes more time, as it include the time to jit the function.

In [61]:
# @title Loss computation (autoregressive loss over multiple steps)
loss, diagnostics = loss_fn_jitted(
    rng=jax.random.PRNGKey(0),
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
print("Loss:", loss)

Loss: 352.90106


In [56]:
# @title Gradient computation (backprop through time)

loss, diagnostics, next_state, grads = grads_fn_jitted(
    inputs=train_inputs,
    targets=train_targets,
    forcings=train_forcings)
mean_grad = np.mean(jax.tree_util.tree_flatten(jax.tree_util.tree_map(lambda x: np.abs(x).mean(), grads))[0])
print(f"Loss: {loss:.4f}, Mean |grad|: {mean_grad:.6f}")


Loss: 352.8958, Mean |grad|: 0.990390


# Run the model

In [48]:
# @title Autoregressive rollout (loop in python)

print("Inputs:  ", eval_inputs.dims.mapping)
print("Targets: ", eval_targets.dims.mapping)
print("Forcings:", eval_forcings.dims.mapping)

predictions = rollout.chunked_prediction(
    run_forward_jitted,
    rng=jax.random.PRNGKey(0),
    inputs=eval_inputs,
    targets_template=eval_targets * np.nan,
    forcings=eval_forcings)
predictions

/home/jupyter-lukas/.conda/envs/MTLMenv/lib/python3.12/site-packages/graphcast/rollout.py:127: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_target_steps = targets_template.dims["time"]


Inputs:   {'batch': 1, 'time': 2, 'lon': 64, 'lat': 32, 'level': 13}
Targets:  {'batch': 1, 'time': 40, 'lon': 64, 'lat': 32, 'level': 13}
Forcings: {'batch': 1, 'time': 40, 'lon': 64, 'lat': 32}


/home/jupyter-lukas/.conda/envs/MTLMenv/lib/python3.12/site-packages/graphcast/rollout.py:205: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_inputs = prev_inputs.dims["time"]


<xarray.Dataset> Size: 27MB
Dimensions:                  (time: 40, batch: 1, lon: 64, lat: 32, level: 13)
Coordinates:
  * lat                      (lat) float64 256B -87.19 -81.56 ... 81.56 87.19
  * level                    (level) int64 104B 50 100 150 200 ... 850 925 1000
  * lon                      (lon) float64 512B 0.0 5.625 11.25 ... 348.8 354.4
  * time                     (time) timedelta64[ns] 320B 0 days 06:00:00 ... ...
Dimensions without coordinates: batch
Data variables:
    10m_u_component_of_wind  (time, batch, lon, lat) float32 328kB -4.875 ......
    10m_v_component_of_wind  (time, batch, lon, lat) float32 328kB 1.147 ... ...
    2m_temperature           (time, batch, lon, lat) float32 328kB 251.3 ... ...
    geopotential             (time, batch, level, lon, lat) float32 4MB 2.019...
    mean_sea_level_pressure  (time, batch, lon, lat) float32 328kB 9.906e+04 ...
    specific_humidity        (time, batch, level, lon, lat) float32 4MB 2.815...
    temperature              (time, batch, level, lon, lat) float32 4MB 234.6...
    total_precipitation_6hr  (time, batch, lon, lat) float32 328kB -0.002876 ...
    u_component_of_wind      (time, batch, level, lon, lat) float32 4MB -3.85...
    v_component_of_wind      (time, batch, level, lon, lat) float32 4MB 1.522...
    vertical_velocity        (time, batch, level, lon, lat) float32 4MB 0.001...

In [62]:
# get the size of the array:
print("size of array:", (predictions.nbytes/ (1024*1024)) ,"MB")

size of array: 25.938636779785156 MB


# Save the evaluation run to file


In [ ]:
# Specify the path where you want to save the Zarr file
zarr_path = "Eval_preds/mse_64x32_2020.zarr"

# Save the dataset to the Zarr file
predictions.to_zarr(zarr_path, mode='w')

# Save the model to file

In [ ]:
# @title saving the model
# Save the model
np.savez("models/model_64x32_mse.npz", **hk.data_structures.to_mutable_dict(params))

Now, our trained model is saved to a file ,which can be used to load and run again.